In [2]:
import os
import pandas as pn
import numpy as np
from tensorflow import set_random_seed
from random import seed
from sys import platform
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, GRU
from keras.utils import np_utils

In [3]:
data_directory = r"D:/CRT_summer_school/data_v_7_stc"  #Указать путь к папке с данными

opensmile_path = r"D:/CRT_summer_school/opensmile-2.3.0"  #Указать путь к распакованному архиву с opensmile

input_train_path = data_directory + r"/audio//"
input_test_path = data_directory + r"/test//"
output_train_path = data_directory + r'/train_features'
output_test_path = data_directory + r"/test_features"

#Создаем дериктории под фичи трейна и теста
if not os.path.exists(output_train_path):
    try:
        os.mkdir(output_train_path)
    except OSError:
        print("Создать директорию %s не удалось" % output_train_path)
    else:
        print("Успешно создана директория %s " % output_train_path)
else:
    print("Директория %s уже существует" % output_train_path)

if not os.path.exists(output_test_path):
    try:
        os.mkdir(output_test_path)
    except OSError:
        print("Создать директорию %s не удалось" % output_test_path)
    else:
        print("Успешно создана директория %s " % output_test_path)
else:
    print("Директория %s уже существует" % output_test_path)

#Считываем названия файлов теста
test_samples = os.listdir(input_test_path)

#Считываем мета-инфу трейна
train_meta = pn.read_csv(data_directory + r"/meta/meta.txt", sep='\t', header=None)

Директория D:/CRT_summer_school/data_v_7_stc/train_features уже существует
Директория D:/CRT_summer_school/data_v_7_stc/test_features уже существует


In [4]:
# В соответствии с ОС делаем шаблон команды на извлечение фич с помощью opensmile
raw_command = ""
if platform == "linux" or platform == "linux2":
    raw_command = r"{0}/bin/linux_x64_standalone_static/SMILExtract -C {0}/config/ComParE_2016.conf ".format(
        opensmile_path)
elif platform == "win32":
    raw_command = r"{0}/bin/Win32/SMILExtract_Release.exe -C {0}/config/ComParE_2016.conf ".format(opensmile_path)

#Извлекаем фичи трейн
for file in train_meta.values:
    if not os.path.exists(output_train_path + "/" + file[0].replace(".wav", ".csv")):
        command = raw_command + "-I " + input_train_path + file[0] + " "
        command += "-D " + output_train_path + "/" + file[0].replace(".wav", ".csv")
        os.system(command)
#Извлекаем фичи тест
for file in test_samples:
    if not os.path.exists(output_test_path + "/" + file.replace(".wav", ".csv")):
        command = raw_command + "-I " + input_test_path + file + " "
        command += "-D " + output_test_path + "/" + file.replace(".wav", ".csv")
        os.system(command)


In [5]:
classes = {'background': 0, 'bags': 1, 'door': 2, 'keyboard': 3, 'knocking_door': 4, 'ring': 5, 'speech': 6, 'tool': 7}

#Перемешиваем тренировочный набор 
shuffled_train_meta = train_meta.sample(frac=1, random_state=777)



In [6]:
#Для нормализации данных находим мин и макс значения фич
maximum_val = []
minimum_val = []
for i in range(len(train_meta)):
    data = pn.read_csv(output_train_path + "/" + train_meta[0].iloc[i].replace(".wav", ".csv"), sep=";")
    data = data[::4]
    maximum_val.append(data.max())
    minimum_val.append(data.min())
    if i % 1000 == 0:
        print(i)

maximum_val = pn.concat(maximum_val, axis=1).transpose().max()
minimum_val = pn.concat(minimum_val, axis=1).transpose().min()

maximum_val.to_csv("maximum_val.csv", sep='\t', encoding='utf-8')
minimum_val.to_csv("minimum_val.csv", sep='\t', encoding='utf-8')

min_val = np.array(minimum_val.drop("name").drop("frameTime"))
max_val = np.array(maximum_val.drop("name").drop("frameTime"))
difference = np.array(max_val - min_val)


0


1000


2000


3000


4000


...

7000


8000


9000


10000


11000


In [7]:
#Нарезаем аудиофайл на куски длиной 2сек (Если файл < 2сек, но > 0.4сек , то заполняем его до 2сек дублируя фичи)
def slice_sample(data):
    ne = []
    data = data[::4]
    for j in range(0, len(data), 50):
        new_sample = data.iloc[j:50 + j]
        if len(new_sample) != 50 and len(new_sample) >= 10:
            extend_mass = []
            for i in range(50 - len(new_sample)):
                extend_mass.append(new_sample.iloc[i % len(new_sample)])
            new_sample = new_sample.append(extend_mass)
        if len(new_sample) == 50:
            ne.append(new_sample.drop("name", 1).drop("frameTime", 1))

    return ne


In [8]:
batch_size = 2000

# Получаем массив нарезанных файлов для подачи в НС
def get_part_of_data(num, batch):
    label = []
    X_cl = np.array([])
    for i in range(batch):
        data = pn.read_csv(output_train_path + '/' + shuffled_train_meta[0].iloc[i + num * batch_size].replace(".wav", ".csv"), sep=';')
        ne = slice_sample(data)
        for _ in range(len(ne)):
            label.append(classes[shuffled_train_meta[4].iloc[i + num * batch_size]])

        for k in range(0, len(ne)):
            try:
                if X_cl.size == 0:
                    X_cl = ne[0].values.reshape(1, ne[0].values.shape[0], ne[0].values.shape[1])
                else:
                    X_cl = np.concatenate((X_cl, ne[k].values.reshape(1, ne[k].values.shape[0], ne[k].values.shape[1])))
            except:
                print("--Concat_error--", k)
        if i % 100 == 0:
            print(i)
    return X_cl, np.array(label)


num_parts = int(len(train_meta) / batch_size)
last_batch = len(train_meta) - num_parts * batch_size + batch_size

print("Всего файлов = "+str(len(train_meta)))
print("Число частей = "+str(num_parts))
print("Длина последней части = "+str(last_batch))

Всего файлов = 11307
Число частей = 5
Длина последней части = 3307


In [10]:
# Нарезаем тестовые данные
Y_test = []
X_test = np.array([])
number_of_sample_parts = [] #Число частей на которые нарезается входной аудиофайл
for i in range(len(test_samples)):
    data = pn.read_csv(output_test_path + "/" + test_samples[i].replace(".wav", ".csv"), sep=';')
    ne = slice_sample(data)
    number_of_sample_parts.append(len(ne))
    flag = False
    for key in list(classes.keys())[::-1]:
        if key in test_samples[i]:
            flag = True
            Y_test.append(classes[key])
            break
    if not flag:
        Y_test.append(len(classes))

    for k in range(0, len(ne)):
        try:
            if X_test.size == 0:
                X_test = ne[0].values.reshape(1, ne[0].values.shape[0], ne[0].values.shape[1])
            else:
                X_test = np.concatenate((X_test, ne[k].values.reshape(1, ne[k].values.shape[0], ne[k].values.shape[1])))
        except:
            print("--Error--", k)
            Y_test.pop()
print(X_test.shape, np.array(Y_test).shape)


(1956, 50, 130) (610,)


In [11]:
X,Y=get_part_of_data(0, batch_size)
Y=np_utils.to_categorical(Y,num_classes=len(classes))
print(X.shape)

0

100


200


300


400


...

1600


1700


1800


1900


(5849, 50, 130)


In [12]:
X1,Y1=get_part_of_data(1, batch_size)
Y1=np_utils.to_categorical(Y1,num_classes=len(classes))
print(X1.shape)

0


100


200


300


400


...

1600


1700


1800


1900


(5698, 50, 130)


In [13]:
X_valid,Y_valid=get_part_of_data(2, 500)
Y_valid=np_utils.to_categorical(Y_valid,num_classes=len(classes))
print(X_valid.shape)


0


100


200


300


400


(1381, 50, 130)


In [14]:
#Нормализация к [-1;1]
Xn = 2*(X-min_val)/difference-1
Xn1 = 2*(X1-min_val)/difference-1
Xn_valid = 2*(X_valid-min_val)/difference-1
Xn_test = 2*(X_test-min_val)/difference-1

In [15]:
#Объединяем в один массив
Xn_train = np.concatenate((Xn, Xn1), axis=0)
Y_train = np.concatenate((Y, Y1), axis=0)
print(Xn_train.shape)
print(Y_train.shape)


(11547, 50, 130)
(11547, 8)


In [16]:
set_random_seed(771)
seed(771)
np.random.seed(771)

model = Sequential()
model.add(GRU(130, activation='tanh', return_sequences=True, input_shape=(Xn_train.shape[1], Xn_train.shape[2])))
model.add(Dropout(0.3))
model.add(GRU(30, activation="tanh", return_sequences=False))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 50, 130)           101790    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 130)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 30)                14490     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 248       
Total params: 116,528
Trainable params: 116,528
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = model.fit(Xn_train, Y_train,batch_size=32,epochs=4, verbose=1,validation_data=(Xn_valid,Y_valid))

Train on 11547 samples, validate on 1381 samples
Epoch 1/4


   32/11547 [..............................] - ETA: 12:28 - loss: 2.2080 - acc: 0.1562

   64/11547 [..............................] - ETA: 6:31 - loss: 2.1249 - acc: 0.1875 

   96/11547 [..............................] - ETA: 4:32 - loss: 2.0985 - acc: 0.1979

  128/11547 [..............................] - ETA: 3:33 - loss: 2.1626 - acc: 0.1797

...

11424/11547 [============================>.] - ETA: 0s - loss: 0.3060 - acc: 0.8988

11456/11547 [============================>.] - ETA: 0s - loss: 0.3057 - acc: 0.8990

11488/11547 [============================>.] - ETA: 0s - loss: 0.3055 - acc: 0.8992

11520/11547 [============================>.] - ETA: 0s - loss: 0.3049 - acc: 0.8995

11547/11547 [==============================] - 38s 3ms/step - loss: 0.3045 - acc: 0.8996 - val_loss: 0.2483 - val_acc: 0.9298


In [18]:
# Записываем модель в файл
model_json = model.to_json()
json_file = open("model.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_weights.h5")

In [19]:
# Загружаем сохраненную модель
json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
from keras.models import model_from_json
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_weights.h5")
loaded_model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

In [20]:
#Собираем вероятности принадлежности к классам всех частей аудиофайла и присваиваем класс с наибольшей вероятностью
predictions = model.predict_on_batch(Xn_test)
i = 0
final_prediction_probability = []
for sample_len in number_of_sample_parts:
    tmp = np.array([0, 0, 0, 0, 0, 0, 0, 0], dtype='float64')
    for _ in range(sample_len):
        tmp += predictions[i]
        i += 1
    final_prediction_probability.append(tmp / sample_len)
final_prediction = [np.argmax(pred) for pred in final_prediction_probability]

#Считаем accuracy
acc = 0
total = 0
for i in range(len(final_prediction)):
    if Y_test[i] != 8:
        total += 1
        if final_prediction[i] == Y_test[i]:
            acc += 1
print("Точность классификации на тестовой выборке %s " % str(acc / total * 100))


Точность классификации на тестовой выборке 90.69767441860465 


In [21]:
my_file = open(data_directory+r"/result.txt", "w")

for i, file in enumerate(test_samples):
     
    my_file.write(file+'\t'+str(max(final_prediction_probability[i]))+'\t'+str(list(classes.keys())[final_prediction[i]])+'\n')

my_file.close()

model.save(filepath=data_directory+r"/model")